<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [2]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# загрузим обучающую и тестовую выборки
# train_df = pd.read_csv("../../data/train_sessions.csv", index_col="session_id")
# test_df = pd.read_csv("../../data/test_sessions.csv", index_col="session_id")
train_df = pd.read_csv('/content/drive/MyDrive/Colab/mlcourse_ai_russian/topic04_linear_models/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('/content/drive/MyDrive/Colab/mlcourse_ai_russian/topic04_linear_models/test_sessions.csv', index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by="time1")

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [5]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# загрузим словарик сайтов
# with open(r"../../data/site_dic.pkl", "rb") as input_file:
    # site_dict = pickle.load(input_file)
with open(r'/content/drive/MyDrive/Colab/mlcourse_ai_russian/topic04_linear_models/site_dic.pkl', 'rb') as input_file:
  site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [6]:
# наша целевая переменная
y_train = train_df["target"]

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [7]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [8]:
from scipy.sparse import csr_matrix

In [9]:
csr_matrix?

In [10]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation).

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [38]:
from sklearn.model_selection import train_test_split


def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=17):
    """
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state
              логистической регрессии
    """

    # Ваш код здесь
    # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=ratio, random_state=seed)

    threshold = int(X.shape[0] * ratio)
    X_train, X_val = X[:threshold], X[threshold:]
    y_train, y_val = y[:threshold], y[threshold:]

    model = LogisticRegression(C=C, random_state=seed)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return roc_auc_score(y_val, y_pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [39]:
# Ваш код здесь
get_auc_lr_valid(np.array(full_sites[:idx_split]), np.array(y_train))

0.5

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [13]:
# функция для записи прогнозов в файл
def write_to_submission_file(
    predicted_labels, out_file, target="target", index_label="session_id"
):
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(1, predicted_labels.shape[0] + 1),
        columns=[target],
    )
    # predicted_df.to_csv(out_file, index_label=index_label)
    predicted_df.to_csv('/content/drive/MyDrive/Colab/mlcourse_ai_russian/topic04_linear_models/' + out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [14]:
# Ваш код здесь
model2 = LogisticRegression(C=1.0, random_state=17)
model2.fit(np.array(full_sites[:idx_split]), np.array(y_train))
y_pred = model2.predict(np.array(full_sites[idx_split:]))

In [15]:
write_to_submission_file(y_pred, 'out.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [16]:
# Ваш код здесь
check = train_df['time1']
start_month = check.apply(lambda x: int(str(x).split()[0].split('-')[0] + str(x).split()[0].split('-')[1]))

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [17]:
# Ваш код здесь
new_df = train_df

scaler = StandardScaler()
start_month_scaled = scaler.fit_transform(np.array(start_month).reshape((-1, 1)))
new_df['Start_month'] = start_month_scaled
new_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site7,time7,site8,time8,site9,time9,site10,time10,target,Start_month
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,0,NaT,0,NaT,0,NaT,0,NaT,0,-1.744405
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,0,NaT,0,NaT,0,NaT,0,NaT,0,-1.744405
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0,-1.744405
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0,-1.744405
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0,-1.744405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12224,50,2014-04-30 23:33:48,50,2014-04-30 23:33:49,48,2014-04-30 23:33:52,49,2014-04-30 23:33:52,48,2014-04-30 23:33:53,...,52,2014-04-30 23:33:54,49,2014-04-30 23:33:54,303,2014-04-30 23:33:57,304,2014-04-30 23:34:00,0,0.681626
164438,4207,2014-04-30 23:34:15,753,2014-04-30 23:34:16,753,2014-04-30 23:34:17,52,2014-04-30 23:34:18,50,2014-04-30 23:34:18,...,3346,2014-04-30 23:35:29,3359,2014-04-30 23:36:12,3346,2014-04-30 23:36:42,38,2014-04-30 23:37:13,0,0.681626
12221,52,2014-04-30 23:38:08,3346,2014-04-30 23:38:10,784,2014-04-30 23:38:13,784,2014-04-30 23:38:18,3346,2014-04-30 23:38:22,...,3324,2014-04-30 23:38:35,7330,2014-04-30 23:38:35,3594,2014-04-30 23:38:35,3329,2014-04-30 23:38:36,0,0.681626


In [18]:
full_sites_train = full_sites[:idx_split]
full_sites_train['Start_month'] = new_df['Start_month']
full_sites_train

<ipython-input-18-9ddf25281f91>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sites_train['Start_month'] = new_df['Start_month']


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,Start_month
session_id,,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0,-1.744405
54843,56,55,56,55,0,0,0,0,0,0,-1.744405
77292,946,946,951,946,946,945,948,784,949,946,-1.744405
114021,945,948,949,948,945,946,947,945,946,946,-1.744405
146670,947,950,948,947,950,952,946,951,946,947,-1.744405
...,...,...,...,...,...,...,...,...,...,...,...
12224,50,50,48,49,48,52,52,49,303,304,0.681626
164438,4207,753,753,52,50,4207,3346,3359,3346,38,0.681626
12221,52,3346,784,784,3346,979,3324,7330,3594,3329,0.681626


In [40]:
get_auc_lr_valid(np.array(full_sites_train), np.array(y_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [20]:
# Ваш код здесь
start_hour = check.apply(lambda x: int(str(x).split()[1].split(':')[0]))
morning = check.apply(lambda x: 1 if int(str(x).split()[1].split(':')[0]) <= 11 else 0)

new_df['Start_hour'] = start_hour
new_df['Morning'] = morning
new_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site8,time8,site9,time9,site10,time10,target,Start_month,Start_hour,Morning
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,0,NaT,0,NaT,0,NaT,0,-1.744405,8,1
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,0,NaT,0,NaT,0,NaT,0,-1.744405,8,1
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0,-1.744405,8,1
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0,-1.744405,8,1
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0,-1.744405,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12224,50,2014-04-30 23:33:48,50,2014-04-30 23:33:49,48,2014-04-30 23:33:52,49,2014-04-30 23:33:52,48,2014-04-30 23:33:53,...,49,2014-04-30 23:33:54,303,2014-04-30 23:33:57,304,2014-04-30 23:34:00,0,0.681626,23,0
164438,4207,2014-04-30 23:34:15,753,2014-04-30 23:34:16,753,2014-04-30 23:34:17,52,2014-04-30 23:34:18,50,2014-04-30 23:34:18,...,3359,2014-04-30 23:36:12,3346,2014-04-30 23:36:42,38,2014-04-30 23:37:13,0,0.681626,23,0
12221,52,2014-04-30 23:38:08,3346,2014-04-30 23:38:10,784,2014-04-30 23:38:13,784,2014-04-30 23:38:18,3346,2014-04-30 23:38:22,...,7330,2014-04-30 23:38:35,3594,2014-04-30 23:38:35,3329,2014-04-30 23:38:36,0,0.681626,23,0


In [21]:
full_sites_train['Start_hour'] = new_df['Start_hour']
full_sites_train['Morning'] = new_df['Morning']
full_sites_train

<ipython-input-21-1d56711409e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sites_train['Start_hour'] = new_df['Start_hour']
<ipython-input-21-1d56711409e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sites_train['Morning'] = new_df['Morning']


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,Start_month,Start_hour,Morning
session_id,,,,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0,-1.744405,8,1
54843,56,55,56,55,0,0,0,0,0,0,-1.744405,8,1
77292,946,946,951,946,946,945,948,784,949,946,-1.744405,8,1
114021,945,948,949,948,945,946,947,945,946,946,-1.744405,8,1
146670,947,950,948,947,950,952,946,951,946,947,-1.744405,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12224,50,50,48,49,48,52,52,49,303,304,0.681626,23,0
164438,4207,753,753,52,50,4207,3346,3359,3346,38,0.681626,23,0
12221,52,3346,784,784,3346,979,3324,7330,3594,3329,0.681626,23,0


In [41]:
# Посчитйте ROC AUC на отложенной выборке для выборки с:

# сайтами, start_month и start_hour
# сайтами, start_month и morning
# сайтами, start_month, start_hour и morning

print(get_auc_lr_valid(np.array(full_sites_train.loc[:, :'Start_hour']), np.array(y_train)))
print(get_auc_lr_valid(np.array(full_sites_train[sites + ['Start_month', 'Morning']]), np.array(y_train)))
print(get_auc_lr_valid(np.array(full_sites_train), np.array(y_train)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5
0.5
0.5


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [42]:
# Ваш код здесь
get_auc_lr_valid(np.array(full_sites_train), np.array(y_train), C=1)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален.

In [ ]:
# Ваш код здесь

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [ ]:
# Ваш код здесь